In [36]:
import json

f = open("data.json", "r")

data = json.load(f)

print(f"{data['tracks']['total']}")

# for i in range(len(data["items"])):
#     print(f"{data['items'][i]['track']['name']}: {data['items'][i]['track']['href']}")

1630


In [98]:
import json
import re
import random
import requests

playlist_token = "BQAHzYs5pFEZ1dzjbE9Ve839ao1PBH1zyPjV3_9oFdip_UySE5CcJ7jRdbXj275d5MQoERYvt_oVC-OyIpzFPL7YC4TGe9iYc4gQpnLQerTx7zBBUC0Mlr4UuaZvDUZwkajwr5qYa07Mzy59IrzzTHhknZL2"
items_token = "BQBPOLDILH5BEUjVv4hRN1-boM0UavaZH35BD979ha0AYwZNuCrf42rrIVfSnaDqjuNfyQJY2weYcNvrGEi1iPD7WU1PO5n1gIs8loY7J2F0-isvTzGNdYmiQ4EKjS3V6d-zM9CD4tuY-lLV3evaxm_-GbVf"

def recieve_playlist_ids():
    
    # removes protocols and other parts of URL not matching an alphanumeric playlist ID i.e "37i9dQZF1DWT6MhXz0jw61"
    pattern = re.compile('^https://open.spotify.com/playlist/([a-zA-Z0-9]+).*$')

    
    playlists = ["https://open.spotify.com/playlist/37i9dQZF1DWT6MhXz0jw61?si=c6c62f9348044ab0", 
                "https://open.spotify.com/playlist/37i9dQZF1DWYmmr74INQlb?si=17c3bf40339e4a4a", 
                "https://open.spotify.com/playlist/0L3CyZ90B6csQ50jb0idrL?si=43c1a6b3bc604182", 
                "https://open.spotify.com/playlist/37i9dQZF1EUMDoJuT8yJsl?si=7665e943ff994818", 
                "https://open.spotify.com/playlist/58NUu1xNbb0AGiovGV0Sl8?si=86b47f62f3e64094"
                ]


    # uses regular expression to match playlist links and extract playlist ID from URL
    playlist_ids = [pattern.findall(play_id)[0] for play_id in playlists]
    return playlist_ids 


def make_request(token, endpoint, fields, offset=0):
    
    # GET request for playlist
    response = requests.get(
        endpoint, # API endpoint
        headers={
            "Authorization": f"Bearer {token}"
        },
        params={
            "fields": fields,
            "limit" : 1,
            "offset" : offset
            }
    )
    return response


# recieves total number of tracks from "Get_Playlist" endpoint
def get_playlist_total():
    totals = []
    api_endpoints = [f"https://api.spotify.com/v1/playlists/{pid}" for pid in recieve_playlist_ids()]
    fields = "tracks.total"
    
    for endpoint in api_endpoints:
        
        response = make_request(playlist_token, endpoint, fields)
            
        json_data = dict(response.json())['tracks']['total']
        totals.append(json_data)
        
    return totals



# uses playlist IDs to call Spotify Web API and save JSON response data
def extract_playlist_json():
    
    response_list = []
    api_endpoints = [f"https://api.spotify.com/v1/playlists/{pid}/tracks" for pid in recieve_playlist_ids()]
    playlist_totals = get_playlist_total()

    print(f"Total: {playlist_totals}")
    
    for i in range(len(api_endpoints)):
         
        # API response specifications and loop parameters
        fields = "items(track(name, href))"
        endpoint = api_endpoints[i]
        offset = 0
        pages = playlist_totals[i]//50
        used_offset = []
        total_requests = 0
        
        print(f"Endpoint: {endpoint}")
       
        
        if pages == 0:
            total_requests = 1
            response = make_request(items_token, endpoint, fields, offset)
            print("REQUEST MADE")
            response_list.append(response.json())
            print(f"Offset used: {offset}, Current page: {offset//50}")
            print(f"Used array: {used_offset}\n")
            print(f"--------------------------\n")
            
        else :
            
            if 0 < pages < 5:
                total_requests = pages
            else:
                total_requests = 5

   
            # Randomise offset to retrieve different results in different API calls
            unique = offset not in used_offset
            while len(used_offset) < total_requests:
                    
                if unique:
                    offset = random.randrange(pages) * 50
                    used_offset.append(offset)

                else:
                    continue
                
                response = make_request(items_token, endpoint, fields, offset)
                print("REQUEST MADE")
                response_list.append(response.json())
                
                print(f"Offset used: {offset}, Current page: {offset//50}")
                print(f"Used array: {used_offset}\n")
                print(f"--------------------------\n")
                
        print(f"Total: {playlist_totals[i]}")  
        print(f"Pages: {pages}")
        print(f"Requests: {total_requests}\n")
                

         # dumps JSON data to response_list for later parsing
        
        
    # print(json.dumps(response_list, sort_keys=True, indent=4))
    return response_list
        
    

recieve_playlist_ids()
get_playlist_total()
tracks=extract_playlist_json()

with open('data.json', 'a') as f:
        json.dump(tracks, f, indent=4)

# with open('data.json', 'r') as read_file:
#     print("Parsed Tracks...\n")
#     playlist_tracks = json.load(read_file)
    
#     print(playlist_tracks['items'])


        

Total: [101, 40, 108, 100, 1652]
Endpoint: https://api.spotify.com/v1/playlists/37i9dQZF1DWT6MhXz0jw61/tracks
REQUEST MADE
Offset used: 50, Current page: 1
Used array: [50]

--------------------------

REQUEST MADE
Offset used: 50, Current page: 1
Used array: [50, 50]

--------------------------

Total: 101
Pages: 2
Requests: 2

Endpoint: https://api.spotify.com/v1/playlists/37i9dQZF1DWYmmr74INQlb/tracks
REQUEST MADE
Offset used: 0, Current page: 0
Used array: []

--------------------------

Total: 40
Pages: 0
Requests: 1

Endpoint: https://api.spotify.com/v1/playlists/0L3CyZ90B6csQ50jb0idrL/tracks
REQUEST MADE
Offset used: 0, Current page: 0
Used array: [0]

--------------------------

REQUEST MADE
Offset used: 0, Current page: 0
Used array: [0, 0]

--------------------------

Total: 108
Pages: 2
Requests: 2

Endpoint: https://api.spotify.com/v1/playlists/37i9dQZF1EUMDoJuT8yJsl/tracks
REQUEST MADE
Offset used: 0, Current page: 0
Used array: [0]

--------------------------

REQUEST MAD